# Windpower

In [1]:
from ruins.core import build_config

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
config, dm = build_config()

In [3]:
climate = dm.read('climate')

## windspeed plot

In [4]:
def windspeeds(climate: xr.Dataset, variable: str, rcp: str = 'rcp85', color='green', bgcolor='lightgreen', fig: go.Figure = None, col: int = 1, row: int = 1) -> go.Figure:
    # get the aggregated data
    df = climate.sel(vars=variable).to_dataframe().groupby(pd.Grouper(freq='a')).mean()
    
    # select by RCP scenario
    if rcp is not None:
        data = df[[c for c in df.columns if c.endswith(rcp)]]
    else:
        data = df

    # get the figure
    if fig is None:
        fig = make_subplots(1, 1)

    # build the basic figure
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=np.nanquantile(data.values, 0.95, axis=1), mode='lines', line=dict(color=bgcolor), fill='none', showlegend=False),
        col=col, row=row
    )
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=np.nanquantile(data.values, 0.05, axis=1), mode='lines', line=dict(color=bgcolor), fill='tonexty', showlegend=False),
        col=col, row=row
    )
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=data.mean(axis=1), mode='lines', line=dict(color=color, width=2), name=f'{rcp.upper()} mean'),
        col=col, row=row
    )

    # layout
    fig.update_layout(
        **{f'yaxis{row}': dict(title=f'{rcp.upper()}<br>Windspeed [m/s]')}
    )

    return fig

In [5]:
fig = make_subplots(3, 1, shared_xaxes=True, vertical_spacing=0.0)

fig = windspeeds(climate, 'u2', rcp='rcp26', fig=fig, row=1, color='blue', bgcolor='lightblue')
fig = windspeeds(climate, 'u2', rcp='rcp45', fig=fig, row=2)
fig = windspeeds(climate, 'u2', rcp='rcp85', fig=fig, row=3, color='red', bgcolor='red')

fig.update_layout(height=600, xaxis3=dict(title='Year'), legend=dict(orientation='h'))

## windenergy upscaling

In [6]:
from typing import Union, Tuple, List
import numpy as np
from numba import jit

TURBINES = dict(
    e53=(0.8, 53),
    e115=(3, 115),
    e126=(7.5, 126)
)

#@jit(forceobj=True)
def turbine_footprint(turbine: Union[str, Tuple[float, int]], unit: str = 'ha'):
    """Calculate the footprint for the given turbine dimension"""
    if isinstance(turbine, str):
        turbine = TURBINES[turbine]
    mw, r = turbine
    
    # get the area - 5*x * 3*y 
    area = ((5 * r) * (3 * r))   # m2

    if unit == 'ha':
        area /= 10000
    elif unit == 'km2':
        area / 1000000

    # return area, mw
    return area, mw

#@jit
def upscale_windenergy(turbines: List[Union[str, Tuple[float, int]]], specs: List[Tuple[float]], site: float = 396.0) -> np.ndarray:
    """
    Upscale the given turbines to the site.
    Pass a list of turbine definitions (either names or MW, rotor_diameter tuples.).
    The function will apply the specs to the site. The specs can either be absolute number of
    turbines per turbine or relative shares per turbine type.
    Returns a tuple for each turbine type.

    Returns
    -------
    List[Tuple[float, int, float]]
        A list of tuples per turbine type. (n_turbines, total_area, total_mw)
    """
    # check input data
    #if not all([len(spec)==len(turbines) for spec in specs]):
    #    raise ValueError('The number of turbines and the number of specs must be equal.')
    
    # result container
    results = np.ones((len(specs) * len(turbines), len(turbines))) * np.NaN

    # get the area and MW for each used turbine type
    turbine_dims = [turbine_footprint(turbine) for turbine in turbines]

    for i in range(len(specs)):
        for j in range(len(turbines)):
            # get the footprint
            #area, mw = turbine_footprint(turbine, unit='ha')
            area, mw = turbine_dims[j]

            # get the available space and place as many turbines as possible
            n_turbines = int((site * specs[i][j]) / area)
            
            # get the used space and total MW
            used_area = n_turbines * area
            used_mw = n_turbines * mw

            results[i * len(turbines) + j,:] = [n_turbines, used_area, used_mw]

    return results


In [7]:
r = upscale_windenergy(['e53', 'e115', 'e126'], [(1, 0, 0), (0., 1, 0.), (0., 0, 1), (0.47, 0.53, 0),])
r

array([[ 93.    , 391.8555,  74.4   ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [ 19.    , 376.9125,  57.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [ 16.    , 381.024 , 120.    ],
       [ 44.    , 185.394 ,  35.2   ],
       [ 10.    , 198.375 ,  30.    ],
       [  0.    ,   0.    ,   0.    ]])

## Structure Windpower data

In [8]:
from ruins.processing.windpower import load_windpower_data
from itertools import product

In [9]:
from typing import List
import warnings
warnings.simplefilter('ignore', category=pd.errors.PerformanceWarning)

import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from ruins.core import DataManager

def windpower_actions_projection(dataManager: DataManager, specs, site: float = 396.0, filter_={}) -> List[pd.DataFrame]:
    """
    """    
    # I guess we have to stick to those here
    turbines=['e53', 'e115', 'e126']

    # handle the specs
    if len(specs) == 1 and any([isinstance(s, range) for s in specs]):
        # there is a range definition
        scenarios = []
        for e1 in specs[0]:
            for e2 in specs[1]:
                for e3 in specs[2]:
                    scenarios.append((e1 / 100, e2 / 100, e3 / 100))
    else:
        scenarios = specs

    # upscale the turbines to the site
    power_share = upscale_windenergy(turbines, scenarios)

    # get the data
    df = load_windpower_data(dataManager)
    # apply filters
    for key, val in filter_.items():
        if key == 'year':
            df = df[val]
        elif key == 'rcp':
            df = df.xs(val, level=1, axis=1)
        elif key == 'gcm':
            df = df.xs(val, level=2, axis=1)

    # aggregate everything
    actions = []
    for i in range(0, len(power_share), len(turbines)):
        data = None
        for j, turbine in enumerate(turbines):
            # get the chunk for this turbine
            chunk = df[turbine.upper(), ].mean(axis=1)  # this is the part I am not sure about

            # multiply with the number of turbines
            chunk *= power_share[i + j][0]

            # merge
            if data is None:
                data = pd.DataFrame(data={turbine: chunk.values}, index=chunk.index)
                #data = chunk
            else:
                #data = pd.merge(data, chunk, left_index=True, right_index=True, how='outer')
                data[turbine] = chunk.values
        actions.append(data)

    return actions

def windpower_distplot(actions: List[pd.DataFrame], fig: go.Figure = None, fill: str = None) -> go.Figure:
    """Plot the actions projected to climate models """    
    if fig is None:
        fig = go.Figure()

    # add all actions
    for i, action in enumerate(actions):
        y = action.sum(axis=1).values
        x = np.linspace(y.min(), y.max(), 100)
        kde = gaussian_kde(y)(x)

        fig.add_trace(
            go.Scatter(x=x, y=kde, mode='lines', line=dict(color='blue', width=0. if fill is not None else 1), fill=fill, showlegend=False)
        )

    return fig


In [36]:
actions = windpower_actions_projection(dm, [(0.47, 0.53, 0)], site=396.0, filter_={'year': slice('2075','2095'), 'rcp': 'rcp85'})

windpower_distplot(actions, fill='tozeroy')

## Terneray Plot

In [13]:
import plotly.figure_factory as ff
from itertools import product
import numpy as np

In [49]:
from ruins.processing.windpower import windpower_actions_projection
def ternary_provision_plot(dataManager: DataManager, filter_: dict = {}, turbines: List[str] = ['e53', 'e115', 'e126'], colorscale: str = 'Cividis', showscale: bool = True) -> go.Figure:
    """Make a ternary plot of the three turbines shares on the axes and the provisioned Windpower as contours"""
    # get amount of turbines what to to if n_turbines != 3
    n_turbines = len(turbines)
    
    # get all combinations
    gen = [np.arange(0.0, 1.0, 0.1) for _ in range(n_turbines)]
    scenarios = [t for t in product(*gen) if abs(sum(t) - 1.0) < 1e-5][1:]

    # get all the actions based on the built scenario 
    actions = windpower_actions_projection(dataManager, scenarios, site=396.0, filter_=filter_)
    print(actions)
    # align data and axis
    data = np.fromiter((action.sum(axis=1).mean() for action in actions), dtype=float)
    axes = np.array([[int(s[i] * 100) for s in scenarios]  for i in range(n_turbines)])

    # build the figure
    fig = ff.create_ternary_contour(axes, data, pole_labels=turbines, ncontours=20, colorscale=colorscale, showscale=showscale)
    return fig

In [50]:
fig = ternary_provision_plot(dm, filter_={'year': slice('2075','2095'), 'rcp': 'rcp85'})
fig

([            e53          e115           e126
time                                        
2075-12-31  0.0  39518.481550  300096.869259
2076-12-31  0.0  39110.958920  295733.849574
2077-12-31  0.0  39504.628598  300812.653535
2078-12-31  0.0  40009.798780  305286.582799
2079-12-31  0.0  39646.470691  302284.774312
2080-12-31  0.0  39224.539771  298293.236770
2081-12-31  0.0  39257.015514  298049.082186
2082-12-31  0.0  39442.267917  300589.012805
2083-12-31  0.0  39320.366286  298604.732242
2084-12-31  0.0  39226.665116  297233.194841
2085-12-31  0.0  39704.789102  302428.863828
2086-12-31  0.0  39311.665627  298915.544807
2087-12-31  0.0  39052.175588  297105.865905
2088-12-31  0.0  39263.665473  298334.803478
2089-12-31  0.0  39463.458120  300689.783668
2090-12-31  0.0  39127.300046  297665.446584
2091-12-31  0.0  38836.629783  296352.847800
2092-12-31  0.0  39119.115068  298828.739763
2093-12-31  0.0  39324.086225  298742.715477
2094-12-31  0.0  39445.700689  300123.495548
2095-12-

AttributeError: 'list' object has no attribute 'sum'